#  Shadow Model

The adversary trains $k$ shadow models $f^i_{shadow}()$, each on a dataset (shadow data) that is similar
in format and distribution as the __target model's private training set__.

Generate shadow data drawing samples from the population where the target model's training data
are drawn from. The shadow model must be __trained in a similar fashion as the target model__.
The larger the number of shadow models, the better the accuracy of the attack model.